In [10]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
import json
import re
import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /home/max/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/max/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/max/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [11]:
with open("./datasets/newdvatch.json", 'r') as file:
    text = json.loads(file.read())


In [12]:
stopwords = nltk.corpus.stopwords.words('russian')
wn = nltk.WordNetLemmatizer()

def rem_stopwords(sentence: list):
  return [word for word in sentence if word not in stopwords]


def is_fully_digits(text):
  return all([char.isdigit() for char in text])

def calc_capital(text):
  percent = sum([1 for symbol in text if symbol.isupper()]) / (len(text) - text.count(" "))
  return round(percent, 3) * 100

  
def calc_punct(text):
  percent = sum([1 for symbol in text if symbol in string.punctuation]) / (len(text) - text.count(" "))
  return round(percent, 3) * 100

def clean_text(text: str):
  # Remove punctuation
  cleaned_punct: str = "".join([symbol for symbol in text if symbol not in string.punctuation])
  
  # Implement tokenization
  tokenized: list = re.split("\W+", text.lower())

  cleaned_fully_digits = [word for word in tokenized if not is_fully_digits(word)]

  # Remove stopwords & Lemmatize
  nostopwords_and_lemmatized = [wn.lemmatize(word) for word in cleaned_fully_digits if word not in stopwords]
  # lemmatized = [wn.lemmatize(word) for word in cleaned_fully_digits]

  return nostopwords_and_lemmatized

In [13]:
msgs = []

for index, message in enumerate(text['messages']):
    
    if type(message['text']) == list:
        
        joined_text = "".join([m_text.replace("Обсуждение", "") for m_text in message['text'] if type(m_text) == str])
        msgs.append(joined_text)
    
    elif type(message['text']) == str:
        if len(message['text']):
            msgs.append(message['text'].replace("Обсуждение", ""))

In [14]:
len(msgs)

30995

In [15]:
df = pd.DataFrame({"message": msgs})
df

,message
0,На юге Москвы вспыльчивый водитель едва не поб...
1,Мужчина избил молотком свою жену-кассира прямо...
2,"Лидер ЛДПР Владимир Жириновский считает, что С..."
3,"Марк Чепмен, убийца участника группы The Beatl..."
4,"— это официальный канал Двача - , здесь будут..."
...,...
30990,"«», — сообщил один из жителей дома.\n\nВсё про..."
30991,Вышел трейлер шестого сезона Релиз 4 сентября.
30992,Находку сделали при раскопках средневекового п...
30993,— заявил чиновник Госдепа (Reuters).


In [16]:
count_vect = CountVectorizer(ngram_range=(1,1),  max_features=10000, analyzer=clean_text)
X = count_vect.fit_transform(df.message)
X

<30995x10000 sparse matrix of type '<class 'numpy.int64'>'
	with 420397 stored elements in Compressed Sparse Row format>

In [17]:
tokenized_df = pd.DataFrame(X.toarray(), columns=count_vect.get_feature_names())
tokenized_df.head()

/home/max/.local/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,5g,a,air,airline,aka,amazon,app,apple,audi,b,...,японии,япония,японская,японской,ярмыш,ярослав,ярославской,ясно,яшина,ё
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
